### [12.4 텐서플로 함수와 그래프](https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/function.ipynb?hl=ko)

텐서플로2는 그래프 사용이 매우 간단하고 쉬움  
  
### '그래프'란?  
- tf.Operation 객체의 집합을 포함하고 있는 데이터 구조로서, 연산의 단위와 텐서 객체, 연산간에 흐르는 데이터 단위를 나타냄.

### tf 1.x VS tf 2.x  
**tf 1.x**: 연산그래프를 생성하고, 연산 수행전 그래프로 부터 텐서 수행 순서 최적화 진행을 모두 결정한 후에야 비로소 **게으르게 연산을 수행**.  또한 추가적으로 Session을 열어서 그래프를 명시적으로 작동 시켜야 됨  
  
  
**tf 2.x**: **즉시 실행**모드로 바뀜.  

---
**AutoGraph 와 tf.function 데코레이터의 필요성**
- 만약, 복잡한 모델의 성능 최적화가 필요하다거나, 다른 장비로 모델을 내보내야 하는 등의 경우에 파이썬 **즉시 실행** 코드를 TensorFlow Graph 로 변환해야할 필요가 있음!      
이런 경우 AutoGraph 와 tf.function 데코레이터를 사용함!  
  
-- TensorFlow 1.x 버전때처럼 Placeholder를 정의하거나 Session을 열고 작동시켜야하는 번거로움

---

In [2]:
def cube(x):
    return x**3

정수나 실수 같은 파이썬 상수 or 텐서를 사용하여 함수 호출 가능!

In [3]:
cube(2)

8

In [4]:
import tensorflow as tf
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

#### 1. tf.function()  
- 파이썬 함수를 텐서플로 함수로 바꿔주는 함수  

   ★ 텐서플로 함수를 쓰는 이유??  
     결론: 원본 파이썬 함수 보다 **훨씬 빠르게** 실행됨   
     텐서플로는 사용하지 않는 노드는 제거하고, 표현을 **단순화** 하여 계산 **그래프를 최적화**함. 최적화된 그래프가 준비되면 텐서플로함수는 적절한 순서에 맞추어 그래프 내의 연산을 효율적으로 수행하게 됨  
     
      
- 원래 파이썬 함수처럼 사용할 수 있으며,타입이 tensor인 동일한 결과값을 반환함. 

In [5]:
tf_cube = tf.function(cube)
tf_cube

In [6]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [8]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

참고) 내부적으로 tf.function()은 cube() 함수에서 수행되는 **계산을 분석**하고, 동일한 작업을 수행하는 **계산 그래프를 생성**함!     

→ 뒤쪽 내용 확인!  

---

#### 2) [tf.function 데코레이터](https://rfriend.tistory.com/555)
- 파이썬 함수를 tf 2.x 버전에서 텐서를 inout으로 받아서, 그래프 최적화 하여 연산 수행을 가능하게 하는 장치

In [10]:
@tf.function  
def calc(a,b,c):
    d = a * b * c
    e = a * b * c
    return d,e

아래와 같이 작성 해주면, python 사용자 정의 함수가 AutoGraph에서 어떻게 변환이 되었는지 바로 확인 할 수 있음

In [11]:
print(tf.autograph.to_code(calc.python_function))

def tf__calc(a, b, c):
    with ag__.FunctionScope('calc', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        d = ((ag__.ld(a) * ag__.ld(b)) * ag__.ld(c))
        e = ((ag__.ld(a) * ag__.ld(b)) * ag__.ld(c))
        try:
            do_return = True
            retval_ = (ag__.ld(d), ag__.ld(e))
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



결과적으로, tf 1.x에서는 Session을 열고 명시적으로 그래프 연산을 시켜줘야 한다면,   
tf 2.x 버전에서는 Session없이 바로 그래프 연산을 수행할 수 있음  
  
- Session없이 바로 위에서 정의하고, 그래프로 변환했던 calc() 함수에 a,b,c 상수 텐서를 입력하여 d,e 텐서를 결과로 반환할 수 있음 (바로 아래코드)

In [14]:
# input constant tensor
a = tf.constant(2.0, name="a")
b = tf.constant(3.0, name="b")
c = tf.constant(4.0, name="c")

# run AutoGraph function without opening Session()
d, e = calc(a, b, c)

print('d:', d)
print('e:', e)

d: tf.Tensor(24.0, shape=(), dtype=float32)
e: tf.Tensor(24.0, shape=(), dtype=float32)


---
#### 3. 텐서플로 함수  
- 입력 데이터의 크기, 타입에 맞추어 그래프를 생성함.  
- 텐서플로 함수는 이러한 다형성(다양한 매개변수 타입과 크기)을 가짐.  
  단, 매개변수로 **텐서**를 사용했을 경우에만! 파이썬 값을 함수에 전달하면, 고유한 값마다 새로운 그래프가 만들어짐  
    
참고)  
- 파이썬 값으로 텐서플로 함수 여러번 호출 시, **프로그램이 느려지고 메모리 사용이 많아짐!**  
따라서 몇개의 고유한 값이 있는 경우에만 매개변수 값에 파이썬 값을 사용해야함!

In [16]:
tf_cube(tf.constant(10)) # []크기의, int32에 맞는 그래프 생성

<tf.Tensor: shape=(), dtype=int32, numpy=1000>

In [17]:
tf_cube(tf.constant(20)) # 위에서 만든 그래프와 동일한 그래프가 재사용됨.

<tf.Tensor: shape=(), dtype=int32, numpy=8000>

In [18]:
tf_cube(tf.constant([10,20])) # [2] 크기의 int32텐서에 맞는 새로운 그래프가 생성됨

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1000, 8000])>

---  
### 12.4.1 오토그래프와 트레이싱

텐서플로우의 그래프 생성단계  
    
#### step1. 오토그래프 생성  
- 파이썬 함수의 소스 코드 분석: while, if, return문 등 제어문을 모두 찾음  
  (파이썬이 제어문을 찾을 수 있는 방법을 제공하지 않기 때문에, 수행함)  
  
#### step2. 제어문 변환, 업그레이드 버전 생성
- 오토그래프는 찾은 제어문을 텐서플로 연산으로 바꾼 업그레이드 된 버전을 만듦.   
Ex) 반복문 → tf.while_loop(), sum_squares() → tf__sum_squares() 등등
  
#### step3.  업그레이드 함수 호출  
- 매개변수 값 전달 대신, **심볼릭 텐서**를 전달.    
cf) 심볼릭텐서: 실제 값이 없고 이름, 데이터타입, 크기만 가지는 텐서  
  
  Ex) sum_squares(tf.constant(10)) 호출 → tf__sum_squares()함수는 크기가 [ ]이고 int32타입의 심볼릭 텐서를 사용해 호출됨.  
- 각 텐서플로 연산이 해당 연산을 나타내게 되고, 텐서출력을 위해 그래프에 노드가 추가되어. 그래프모드로 실행되어짐   
  
#### step4. 최종 그래프 생성 : 트레이싱 과정  
- 노드는 연산을 나타내고, 화살표는 텐서를 나타냄.

--- 
### 12.4.2 텐서플로 함수 사용 방법

지켜야할 몇가지 규칙!

#### 1. 넘파이나, 표준 라이브러리를 포함하여 다른 라이브러리 호출 시, 이 부분은 그래프에 포함되지 않음!  
- 실제 텐서플로 그래프는 **텐서플로 구성요소**만 포함할 수 있음.
- np.sum → tf.reduce_sum() / sorted() → tf.sort() 사용  
    
    
#### 2. 다른 파이썬 함수나, 텐서플로 **함수를 호출**할 수 있음.  이런 함수들은 별도로 @tf.function 데코레이터를 적용할 핗요가 없음.  
- 즉, 그래프 모드로 계산할 첫 번째 함수에만 적용하면됨.  
- 다른 함수 호출은 자동으로 그래프 모드가 적용이 됨.  
  
#### 3. 텐서플로 함수에서 텐서플로 변수를 만든다면, 처음 호출 될때만 수행되어야 함.  
- 일반적으로, 텐서플로 함수 밖에서 변수를 생성하는 것이 좋음  
- 변수에 새로운 값을 할당한다면, = 대신 assign()메서드를 사용하기  
  
#### 4. 파이썬 함수의 소스코드는, 텐서플로에서 사용가능 해야함.  
- 사용이 불가능 하다면, 그래프 생성에 실패하거나, 일부 기능을 사용할 수 없을 것임.  
  
#### 5. 텐서플로는 텐서나 데이터셋을 순회하는 for문 만을 감지함  
- for i in range(x) → for i in tf.range(x)  
  
#### 6. 성능면에서는 반복문보다 벡터화된 구현을 사용하는 것이 좋음